In [2]:
!nvidia-smi

Sun May 26 13:46:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   42C    P5             16W /  160W |     320MiB /  16376MiB |     32%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
batch_size = 64
block_size = 256

learning_rate = 3e-4
device = 'cuda'
eval_iters = 200
train_split = 0.9

n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [5]:
with open ('/data/news.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: map string to char ids
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: map integer ids to a string

data = torch.tensor(encode(text), dtype=torch.long)

# Splitting the dataset into training and validation sets
n = int(train_split * len(data)) # 90% training, 10% validation
train_data, validation_data = data[:n], data[n:]

def get_batch(split):
  # batch of data of inputs x and targets y
  data = train_data if split == 'train' else validation_data
  # array(batch_size) of random offsets within data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Context and target sequences
  # ? Why a single sequence is not used?
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y

In [6]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()

  for split in ['train', 'validation']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()

    out[split] = losses.mean()

  model.train()
  return out

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class Head(nn.Module):
  """ One self-attention head"""
  def __init__(self, head_size):
    super().__init__()

    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape

    k = self.key(x)
    q = self.query(x)

    # **0.5 is to avoid softmax from getting too peaky
    # when incoming values are large/spread out.
    wei = q @ k.transpose(-2, -1) * C**-0.5
    wei = wei.masked_fill(
      self.tril[:T, :T] == 0,
      float('-inf')
    )
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v

    return out

class MultiHeadAttention(nn.Module):
  """ Multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()

    self.heads = nn.ModuleList(
      [Head(head_size) for _ in range(num_heads)]
    )
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):
  """ Linear layer with ReLU activation in between """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd, n_embd * 4),
      nn.ReLU(),
      nn.Linear(n_embd * 4, n_embd),
      nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  """ Transformer block: communication (attention) followed by computation (feedforward) """

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))

    return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()

    # Video says: each token directly reads off the logits for the next token
    # from the lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

    self.blocks = nn.Sequential(
      *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
    )
    self.ln_f = nn.LayerNorm(n_embd)

    # self.blocks = nn.Sequential(
    #   Block(n_embd, n_head=4),
    #   Block(n_embd, n_head=4),
    #   Block(n_embd, n_head=4),
    #   nn.LayerNorm(n_embd),
    # )

    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are (B, T) tensors of integers, Batch, Time
    # Token embedding table will return shape of (Batch, Time, Channel)
    # Batch - number of sequences
    # Time - number of tokens in each sequence
    # Channel - Embeddings vector for a token?
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))

    x = tok_emb + pos_emb
    x = self.blocks(x)       # (B, T, C), transformer blocks
    x = self.ln_f(x)         # (B, T, C), final layer norm
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # Cross entropy expects input to be in a 2D tensor of shape (Batch*Time, Channel)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is a (Batch, Time) tensor of integers, representing current context
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]

      # compute the predictions
      logits, loss = self(idx_cond) # (B, T, C)

      # -1 makes very little sense for a bigram model,
      # as we're essentially throwing away everything except the very last token in a batch
      # to make our prediction.
      # This is done in such a way only to allow for easier transition to an N-gram model later.
      logits = logits[:, -1, :] # (B, C)

      # Probabilities from logits
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append to the currently running context
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
max_iters = 1000
eval_interval = 500

for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f'Iter {iter}, Train loss: {losses["train"]}, Validation loss: {losses["validation"]}')

  # Sample batch of data
  xb, yb = get_batch('train')

  # Eval
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

Iter 0, Train loss: 1.282174825668335, Validation loss: 1.3260213136672974
Iter 500, Train loss: 1.2670022249221802, Validation loss: 1.3083372116088867


In [23]:
torch.tensor(encode('Apple'), dtype=torch.long).unsqueeze(0).to(device)

tensor([[34, 80, 80, 76, 69]])

In [27]:
# ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
# encode
ctx = torch.tensor(encode('Apple'), dtype=torch.long).unsqueeze(0).to(device)

batch = model.generate(ctx, max_new_tokens=120)
logits = batch[0].tolist()

print(decode(logits))

Apple revenue beats shortp invend expectations for 20% of its but round
Fire receives new Roles-Fred Airport - Palestinian De
